<center><h1 style="color:grey;"> Generador de Tokens </h1></center>

In [1]:
from IPython.display import clear_output
import gc
import psutil
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

7
Memoria: svmem(total=12649299968, available=7958265856, percent=37.1, used=4691034112, free=7958265856)


In [2]:
import importlib
import dfrtokenuniverse.splitter
import dfrtokenuniverse.word_inventory
import dfrtokenuniverse.constantes


# Recarga librerías propias

In [3]:
importlib.reload(dfrtokenuniverse.splitter)
importlib.reload(dfrtokenuniverse.word_inventory)
importlib.reload(dfrtokenuniverse.constantes)
from dfrtokenuniverse.splitter import TextSplitter
from dfrtokenuniverse.word_inventory import WordInventory
from dfrtokenuniverse.constantes import KDfrNlp
K = KDfrNlp()
splitter = TextSplitter()
invent = WordInventory()

# Ejemplo splitter e inventario

In [4]:
a_dic = {}
for ttkn, n_grama in splitter.split_by_type("Esto es una, hay otra ¿no?\nPues esta es una más ¡SEGURO! 23587,33", 4):
    invent.normalize_and_count(ttkn, n_grama, a_dic)
print(a_dic)
for ttkn in a_dic:
    for n_grama in a_dic[ttkn]:
        p_ngrama = n_grama.replace("\n", "¬")
        print(f"{K.TTKN_DESC[ttkn]} contiene '{p_ngrama}' con {a_dic[ttkn][n_grama]} apariciones")
del a_dic
del ttkn
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

{1: {'esto': 1, 'es': 2, 'una': 2, 'hay': 1, 'otra': 1, 'no': 1, 'pues': 1, 'esta': 1, 'más': 1, 'segu': 1, 'ro': 1}, 2: {' ': 10, ', ': 1, '\n': 1, ',': 1}, 3: {'¿': 1, '?': 1, '¡': 1, '!': 1}, 0: {'2358': 1, '7': 1, '33': 1}}
Letras contiene 'esto' con 1 apariciones
Letras contiene 'es' con 2 apariciones
Letras contiene 'una' con 2 apariciones
Letras contiene 'hay' con 1 apariciones
Letras contiene 'otra' con 1 apariciones
Letras contiene 'no' con 1 apariciones
Letras contiene 'pues' con 1 apariciones
Letras contiene 'esta' con 1 apariciones
Letras contiene 'más' con 1 apariciones
Letras contiene 'segu' con 1 apariciones
Letras contiene 'ro' con 1 apariciones
Separadores contiene ' ' con 10 apariciones
Separadores contiene ', ' con 1 apariciones
Separadores contiene '¬' con 1 apariciones
Separadores contiene ',' con 1 apariciones
Símbolos contiene '¿' con 1 apariciones
Símbolos contiene '?' con 1 apariciones
Símbolos contiene '¡' con 1 apariciones
Símbolos contiene '!' con 1 aparicio

# Recuperamos texto

In [5]:
import pickle
wiki_path = r"D:\datos\wiki_1909627_entradas.pkl"
print(f"Leyendo desde: {wiki_path}")
with open(wiki_path, "rb") as wiki_file:
    wiki_data = pickle.load(wiki_file)
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

Leyendo desde: D:\datos\wiki_1909627_entradas.pkl
0
Memoria: svmem(total=12649299968, available=1996402688, percent=84.2, used=10652897280, free=1996402688)


In [6]:
print(f"Documentos en wiki: {len(wiki_data)}")

Documentos en wiki: 1909627


<center> <h2>Diccionario de n-gramas</h2> </center>

In [7]:
dic_ngramas = {}

In [8]:
saltar  = 0
recoge = 100000 # Entradas de la wikipedia con sus textos
limite = saltar + recoge

<h3>Realizamos varias pasadas para ir acumulando n-gramas</h3>
<hr/>

In [9]:
print(f"Recogemos {recoge} entradas a partir de {saltar}")
cont_entradas = 0
max_len_allowed=35
for entrada in wiki_data:
    if cont_entradas > saltar:
        for ttkn, n_grama in splitter.split_by_type(entrada, max_len=max_len_allowed):
            invent.normalize_and_count(ttkn, n_grama, dic_ngramas)
        for linea in wiki_data[entrada]:
            for ttkn, n_grama in splitter.split_by_type(linea.decode("utf-8"), max_len=max_len_allowed):
                invent.normalize_and_count(ttkn, n_grama, dic_ngramas)
        if cont_entradas > limite:
            break
        if cont_entradas % 1000 == 0:
            clear_output(True)
            print(f"Llevamos {cont_entradas} entradas de {limite} ({round(100*cont_entradas/limite, 2)}%)")
            for ttkn in dic_ngramas:
                print(f"- {len(dic_ngramas[ttkn])} n-gramas recolectados del tipo {K.TTKN_DESC[ttkn]}")
    cont_entradas += 1
salta = limite
limite += recoge

Llevamos 100000 entradas de 100000 (100.0%)
- 573339 n-gramas recolectados del tipo Letras
- 2469 n-gramas recolectados del tipo Separadores
- 10350 n-gramas recolectados del tipo Dígitos
- 410 n-gramas recolectados del tipo Símbolos
- 5744 n-gramas recolectados del tipo Desconocidos
- 3 n-gramas recolectados del tipo Especiales


<hr style="border: 1px dashed #aaa;"/>
<center><h2>Liberamos memoria</h2></center>

In [10]:
print("Memoria:", psutil.virtual_memory())
del wiki_data
print(gc.collect())
print("Memoria:", psutil.virtual_memory())

Memoria: svmem(total=12649299968, available=2215477248, percent=82.5, used=10433822720, free=2215477248)
0
Memoria: svmem(total=12649299968, available=8069984256, percent=36.2, used=4579315712, free=8069984256)


<hr style="color: red; height: 2; border: 3px dotted;"/>
<center><h1 style="color: yellow;">Comienza la fiesta</h1></center>
<hr style="color: red; width: 50%;"/>

In [11]:
max_ngrama_len = 0
max_ngrama_ngrama = ""
max_ngrama_ttkn = ""
maximos = []
for ttkn in dic_ngramas:
    for n_grama in dic_ngramas[ttkn]:
        if len(n_grama) > max_ngrama_len:
            max_ngrama_len = len(n_grama)
            max_ngrama_ngrama = n_grama
            max_ngrama_ttkn = ttkn
    maximos = [n_grama.replace("\n", "¬") for n_grama in sorted(dic_ngramas[ttkn], key=lambda x:dic_ngramas[ttkn][x], reverse=True)[:15]]
    print(f"Para el tipo {K.TTKN_DESC[ttkn].rjust(12)} tenemos {str(len(dic_ngramas[ttkn])).rjust(11)}: {maximos}")
print(f"El ngrama más largo, con {max_ngrama_len} es del tipo {K.TTKN_DESC[ttkn]} ({ttkn}): <|{max_ngrama_ngrama.encode()}|>")
del maximos
print(f"Limpiando caché de {gc.collect()} objetos...")
print("Memoria:", psutil.virtual_memory())

Para el tipo       Letras tenemos      573341: ['de', 'la', 'en', 'el', 'y', 'a', 'que', 'los', 'del', 'se', 'por', 'un', 'con', 'una', 'las']
Para el tipo  Separadores tenemos        2469: [' ', ', ', '. ', ' , ', '.', ' . ', '  ', '-', '.  ', ' .', ' "', ': ', ':', ',', '; ']
Para el tipo      Dígitos tenemos       10350: ['1', '2', '3', '4', '5', '0', '8', '6', '10', '7', '9', '000', '20', '2010', '12']
Para el tipo     Símbolos tenemos         410: ['(', ')', '%', '{', '[', ']', '}', 'º', '>', '&', '$', '#', 'ª', '?', '!']
Para el tipo Desconocidos tenemos        5744: ['«', '»', '²', '”', '“', '—', '°', '–', '=', '′', '″', 'ö', 'ō', 'а', '’']
Para el tipo   Especiales tenemos           3: ['\t', '\t\t', '\t\t\t']
El ngrama más largo, con 35 es del tipo Especiales (4): <|b'cliominervareflexionestrayectoriaes'|>
Limpiando caché de 0 objetos...
Memoria: svmem(total=12649299968, available=8093806592, percent=36.0, used=4555493376, free=8093806592)


# Contabilizacion de monogramas

In [12]:
prob_monogramas = {ttkn:{} for ttkn in dic_ngramas}
acum_monogramas = {ttkn:0 for ttkn in dic_ngramas}
for ttkn in dic_ngramas:
    print(f"Contando tipo {K.TTKN_DESC[ttkn].rjust(12)}...", end="")
    for n_grama in dic_ngramas[ttkn]:
        for monograma in n_grama:
            invent._dict_counter(ttkn, monograma, prob_monogramas)
            acum_monogramas[ttkn] += 1
    print(f" {str(acum_monogramas[ttkn]).rjust(11)} apariciones (probabilidad media: {round(100 * len(prob_monogramas[ttkn]) / acum_monogramas[ttkn], 5)}%)")
print("Probabilidades...")
prob_monogramas = {ttkn:{monograma: prob_monogramas[ttkn][monograma] / acum_monogramas[ttkn] for monograma in prob_monogramas[ttkn]} for ttkn in prob_monogramas}
print("¡Terminado!")
del acum_monogramas
print(f"Limpiando caché de {gc.collect()} objetos...")
print("Memoria:", psutil.virtual_memory())

Contando tipo       Letras...     4727835 apariciones (probabilidad media: 0.00072%)
Contando tipo  Separadores...       14553 apariciones (probabilidad media: 0.08933%)
Contando tipo      Dígitos...       49053 apariciones (probabilidad media: 0.02039%)
Contando tipo     Símbolos...        1220 apariciones (probabilidad media: 1.96721%)
Contando tipo Desconocidos...        5744 apariciones (probabilidad media: 100.0%)
Contando tipo   Especiales...           6 apariciones (probabilidad media: 16.66667%)
Probabilidades...
¡Terminado!
Limpiando caché de 0 objetos...
Memoria: svmem(total=12649299968, available=8090357760, percent=36.0, used=4558942208, free=8090357760)


probabilidad de monogramas

In [13]:
for ttkn in prob_monogramas:
    maximos = [monograma.replace("\n", " ") for monograma in sorted(prob_monogramas[ttkn], key=lambda x: prob_monogramas[ttkn][x], reverse=True)[:10]]
    print(f"Para {K.TTKN_DESC[ttkn].rjust(15)}: {', '.join([f'<{x}>: {round(prob_monogramas[ttkn][x], 7)}' for x in maximos])}")

Para          Letras: <a>: 0.1240655, <e>: 0.0912153, <i>: 0.0836994, <o>: 0.0755189, <r>: 0.0742742, <n>: 0.0677342, <s>: 0.0664067, <t>: 0.052289, <l>: 0.0496308, <c>: 0.0432966
Para     Separadores: < >: 0.5438741, <.>: 0.1528894, <">: 0.0628736, <,>: 0.0507799, <'>: 0.0316773, <->: 0.027898, <:>: 0.0261115, <;>: 0.0199272, < >: 0.5438741, <_>: 0.0172473
Para         Dígitos: <0>: 0.1473101, <1>: 0.1326932, <2>: 0.1122459, <3>: 0.0959778, <4>: 0.0929403, <5>: 0.0896581, <6>: 0.0869875, <7>: 0.0828084, <9>: 0.0803009, <8>: 0.0790777
Para        Símbolos: <}>: 0.2696721, <)>: 0.1114754, <(>: 0.0827869, <{>: 0.0819672, <&>: 0.0778689, <]>: 0.0532787, <!>: 0.0491803, <?>: 0.0434426, <[>: 0.0344262, <|>: 0.0344262
Para    Desconocidos: <ɾ>: 0.0001741, <ˈ>: 0.0001741, <ː>: 0.0001741, <ž>: 0.0001741, <č>: 0.0001741, <​>: 0.0001741, <«>: 0.0001741, <»>: 0.0001741, <“>: 0.0001741, <”>: 0.0001741
Para      Especiales: <	>: 1.0


## Combinatorias

In [14]:
fragmentos = {ttkn: {} for ttkn in dic_ngramas}
fragmentos_n_grama = {ttkn: {} for ttkn in dic_ngramas}
max_len_frag = 5
print(f"Fragmentando ...".rjust(50) + "".join(["|"]*100))
for ttkn in dic_ngramas:
    print(f"Fragmentando {len(dic_ngramas[ttkn])} n-gramas de {K.TTKN_DESC[ttkn]}".rjust(50), end="")
    cada = 1+ len(dic_ngramas[ttkn]) // 100
    cont = 0
    for n_grama in dic_ngramas[ttkn]:
        for l_n in range(2, min([len(n_grama), max_len_frag]) - 1):
            for i_ini in range(len(n_grama) - l_n):
                if n_grama[i_ini:i_ini+l_n] in fragmentos[ttkn]:
                    fragmentos[ttkn][n_grama[i_ini:i_ini+l_n]] += 1
                    if n_grama in fragmentos_n_grama[ttkn][n_grama[i_ini:i_ini+l_n]]:
                        fragmentos_n_grama[ttkn][n_grama[i_ini:i_ini+l_n]][n_grama] += 1
                    else:
                        fragmentos_n_grama[ttkn][n_grama[i_ini:i_ini+l_n]][n_grama] = 1
                else:
                    fragmentos[ttkn][n_grama[i_ini:i_ini+l_n]] = 1
                    fragmentos_n_grama[ttkn][n_grama[i_ini:i_ini+l_n]] = {n_grama: 1}
        cont += 1
        if cont % cada == 0:
            print(".", end="")
    print("!")
print(f"Limpiando caché de {gc.collect()} objetos...")
print("Memoria:", psutil.virtual_memory())

                                  Fragmentando ...||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
            Fragmentando 573341 n-gramas de Letras...................................................................................................!
         Fragmentando 2469 n-gramas de Separadores..................................................................................................!
            Fragmentando 10350 n-gramas de Dígitos...................................................................................................!
             Fragmentando 410 n-gramas de Símbolos..................................................................................!
        Fragmentando 5744 n-gramas de Desconocidos...................................................................................................!
             Fragmentando 3 n-gramas de Especiales...!
Limpiando caché de 0 objetos...
Memoria: svmem(total=1264

## Fragmentos óptimos

In [15]:
acum_frag_n_grama = {ttkn: sum([len(fragmentos_n_grama[ttkn][x]) for x in fragmentos_n_grama[ttkn]]) for ttkn in fragmentos_n_grama}
acum_fragmentos = {ttkn: sum([fragmentos[ttkn][x] for x in fragmentos[ttkn]]) for ttkn in fragmentos}
print(f"Limpiando caché de {gc.collect()} objetos...")
print("Memoria:", psutil.virtual_memory())

Limpiando caché de 0 objetos...
Memoria: svmem(total=12649299968, available=7893495808, percent=37.6, used=4755804160, free=7893495808)


In [16]:
usados = {}
excedentes = {}
n_grama = "Esto es una prueba"
evaluados = ["Es", "to", "ta", "es", "una", "pru", "eba", ".", " "]
n_grama = "Esto es una prueba"
invent.explota_fragmento(n_grama, usados, excedentes, evaluados)
n_grama = "Esto es otra prueba"
invent.explota_fragmento(n_grama, usados, excedentes, evaluados)
print("usados", usados)
print("excedentes", excedentes)
del usados
del evaluados
del excedentes
print(f"Limpiando caché de {gc.collect()} objetos...")
print("Memoria:", psutil.virtual_memory())

usados {'Es': 2, 'to': 2, 'es': 2, ' ': 5, 'una': 1, 'pru': 2, 'eba': 2}
excedentes {'otra': 1}
Limpiando caché de 0 objetos...
Memoria: svmem(total=12649299968, available=7890378752, percent=37.6, used=4758921216, free=7890378752)


## Evaluador de fragmentos en n-gramas para obtener tokens

In [ ]:
def sorted_ttkn(a_ordenar):
    global framentos, acum_fragmentos, fragmentos_n_grama, acum_frag_n_grama
    return sorted(
        a_ordenar,
        key=lambda x: (
            0.55 + len(x) * (1.0 + len(x) - len(set(x))) / max_len_frag
        ) ** (
            (
                1.0 * (fragmentos[ttkn][x] / acum_fragmentos[ttkn]) +
                3.0 * (len(fragmentos_n_grama[ttkn][x]) /acum_frag_n_grama[ttkn])
            ) / 4.0
        ),
        reverse=True
    )

In [ ]:
fragmentos_usados_ttkn = {ttkn: {} for ttkn in fragmentos}
excedentes_ttkn = {ttkn: {} for ttkn in fragmentos}
fragmentos_evaluados_ttkn = {}
print(f"Evaluando fragmentos sobre tokens ...".rjust(70) + "".join(["|"]*100))
for ttkn in dic_ngramas:
    print(f"Ordenando {len(fragmentos[ttkn])} fragmentos de {K.TTKN_DESC[ttkn]}...", end="")
    fragmentos_usados = {}
    excedentes = {}
    previos = sorted_ttkn(fragmentos[ttkn])
    print(f"Fragmentos previos: {len(previos)} = {previos[:10]}")
    evaluados = []
    cont = 0
    cada = 1 + len(previos) // 100
    print(f"Optimizando {len(previos)} n-gramas de {K.TTKN_DESC[ttkn]} cada {cada} un punto.".rjust(70), end="")
    no_esta = True
    for fragmento in previos:
        no_esta = True
        for x in evaluados:
            if len(x) < len(fragmento) and x in fragmento:
                no_esta = False
                break
        if no_esta:
            evaluados.append(fragmento)
        cont += 1
        if cont % cada == 0:
            print(".", end="")
    del previos
    print("!")
    print(f"Fragmentos evaluados: {len(evaluados)} = {evaluados[:10]}")
    cada = 1 + len(dic_ngramas[ttkn]) // 100
    print(f"Evaluando {len(dic_ngramas[ttkn])} n-gramas de {K.TTKN_DESC[ttkn]} cada {cada} un punto.".rjust(70), end="")
    cont = 0
    for n_grama in dic_ngramas[ttkn]:
        if isinstance(n_grama, str) and len(n_grama) < 50:
            invent.explota_fragmento(n_grama, fragmentos_usados, excedentes, evaluados)
        cont += 1
        if cont % cada == 0:
            print(".", end="")
    print("!")
    print("Recogiendo datos...")
    fragmentos_usados_ttkn[ttkn] = fragmentos_usados.copy()
    print(f"Fragmentos usados en {K.TTKN_DESC[ttkn]}: {len(fragmentos_usados)}")
    excedentes_ttkn[ttkn] = excedentes.copy()
    print(f"Excedentes en {K.TTKN_DESC[ttkn]}: {len(excedentes)}")
    del fragmentos_usados
    del excedentes
    print(f"Limpiando caché de {gc.collect()} objetos...")
print(f"Limpiando caché de {gc.collect()} objetos...")
print("Memoria:", psutil.virtual_memory())

                                 Evaluando fragmentos sobre tokens ...||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Ordenando 19731 fragmentos de Letras...Fragmentos previos: 19731 = ['ll', 'ara', 'ill', 'rr', 'ell', 'ada', 'lla', 'ala', 'arr', 'ana']
               Optimizando 19731 n-gramas de Letras cada 198 un punto....................................................................................................!
Fragmentos evaluados: 18676 = ['ll', 'ara', 'rr', 'ada', 'ala', 'ana', 'aba', 'ili', 'ama', 'ss']
               Evaluando 573341 n-gramas de Letras cada 5734 un punto....................................................................................................!
Recogiendo datos...
Fragmentos usados en Letras: 14100
Excedentes en Letras: 37
Limpiando caché de 0 objetos...
Ordenando 468 fragmentos de Separadores...Fragmentos previos: 468 = ['   ', '  ', '...', '.  ', ' . ', '..', ' , ', ',  ', '  .', ' ..']
    

## Guardado (si procede)

In [21]:
fragmentos_usados_path = r"D:\datos\tokens_usados.pkl"
excedentes_path = r"D:\datos\tokens_excedentes.pkl"
print(f"Guardando {fragmentos_usados_path}")
with open(fragmentos_usados_path, "wb") as file:
    pickle.dump(fragmentos_usados_ttkn, file)
print(f"Guardando {excedentes_path}")
with open(excedentes_path, "wb") as file:
    pickle.dump(excedentes_ttkn, file)

Guardando D:\datos\tokens_usados.pkl
Guardando D:\datos\tokens_excedentes.pkl


# Diccionario de tokens

## Orden y numeración

In [22]:
ttkn_start_id = {
    K.TTKN_ESP: 50,
    K.TTKN_SEP: 1000,
    K.TTKN_SIM: 5000,
    K.TTKN_DIG: 20000,
    K.TTKN_LET: 40000,
    K.TTKN_UNK: 90000
}

## Generación de tokens

In [ ]:
# Empezamos por los tokens del sistema
tokens = {}
tokens[K.TTKN_SIS] = {token:K.MAIN_TOKEN_DICT[token] for token in K.MAIN_TOKEN_DICT}
# Inicializamos los tipos de token previendo su rellenado posterior
for ttkn in fragmentos:
    tokens[ttkn] = {}
# Iniciamos desde el token_id = 10 como minimo
token_id = 10
# A rellenar...
for ttkn in ttkn_start_id:
    if ttkn not in fragmentos_usados_ttkn:
        continue
    # Situamos la numeracion optimizada
    if token_id < ttkn_start_id[ttkn]:
        token_id = ttkn_start_id[ttkn]
    else:
        token_id += ttkn_start_id[ttkn]
    print(f"Para {K.TTKN_DESC[ttkn]} empezamos por {token_id}", end=",")
    # Incluimos en esa trancha de ids los tokens
    for fragmento in sorted_ttkn(fragmentos_usados_ttkn[ttkn]):
        tokens[ttkn][fragmento] = token_id
        token_id += 1
    print(f" usados ({token_id-1}),", end="")
    for excedente in excedentes_ttkn[ttkn]:
        tokens[ttkn][excedente] = token_id
        token_id += 1
    print(f" excedentes ({token_id-1}),", end="")
    # Que no se nos olviden los monogramas encontrados
    if ttkn in prob_monogramas:
        for monograma in sorted(prob_monogramas[ttkn], key=lambda x: prob_monogramas[ttkn][x], reverse=True):
            if monograma not in tokens[ttkn]:
                tokens[ttkn][monograma] = token_id
                token_id += 1
    print(f" monogramas ({token_id-1}),", end="")
    # Para que quede todo relleno, los de las constantes
    if ttkn in K.TIPOS_TKN:
        for caracter in K.TIPOS_TKN[ttkn]:
            if caracter.lower() not in tokens[ttkn]:
                tokens[ttkn][caracter.lower()] = token_id
                token_id += 1
    print(f" y constantes. Terminamos por {token_id-1}. Tenemos {len(tokens[ttkn])}")
    print(f"Llevamos {sum([len(tokens[x]) for x in tokens])}")


Para Especiales empezamos por 50, usados (49), excedentes (52), monogramas (52), y constantes. Terminamos por 57. Tenemos 8
Llevamos 17
Para Separadores empezamos por 1000, usados (1328), excedentes (1357), monogramas (1357), y constantes. Terminamos por 1358. Tenemos 359
Llevamos 376
Para Símbolos empezamos por 5000, usados (5059), excedentes (5226), monogramas (5226), y constantes. Terminamos por 5226. Tenemos 227
Llevamos 603
Para Dígitos empezamos por 20000, usados (20909), excedentes (20919), monogramas (20919), y constantes. Terminamos por 20919. Tenemos 920
Llevamos 1523
Para Letras empezamos por 40000, usados (54099), excedentes (54136), monogramas (54136), y constantes. Terminamos por 54136. Tenemos 14137
Llevamos 15660
Para Desconocidos empezamos por 90000, usados (89999), excedentes (95743), monogramas (95743), y constantes. Terminamos por 95743. Tenemos 5744
Llevamos 21404


# Guardado de Tokens

In [20]:
tokens_path = r"D:\datos\tokens_ttkn_id_v20241208.pkl"
print(f"Guardando {tokens_path}")
with open(tokens_path, "wb") as file:
    pickle.dump(tokens, file)

Guardando D:\datos\tokens_ttkn_id_v20241208.pkl
